<a href="https://colab.research.google.com/github/AsmaeKarmouchi/MiniGPT-FromScratch/blob/main/Tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let's build the GPT Tokenizer


# Résumé : Rôle et problèmes du Tokenizer dans les LLM

## 1. Qu’est-ce qu’un Tokenizer ?
Le Tokenizer est une étape séparée et indispensable des modèles de langage (LLMs).  
Il sert à convertir :
- `encode()` : texte → tokens (entiers)
- `decode()` : tokens → texte

Les modèles comme GPT **ne travaillent jamais directement avec du texte**, mais uniquement avec des tokens.

---

## 2. Un système indépendant du modèle
Le tokenizer :
- a son **propre jeu d’entraînement**,
- utilise **son propre algorithme** (souvent Byte Pair Encoding — BPE),
- est **entraîné séparément**,
- n’est pas un réseau neuronal.

Une fois entraîné, il ne change plus.  
→ Le LLM doit s’adapter aux tokens fournis, même s’ils sont imparfaits.

---

## 3. Comment fonctionne BPE ?
Le Byte Pair Encoding construit un vocabulaire de sous-mots :
1. commence avec des caractères,
2. fusionne les paires les plus fréquentes,
3. continue jusqu’à obtenir des milliers de morceaux optimaux.

Exemples :
- "playing" → "play" + "ing"
- "unbelievable" → "un" + "bel" + "ievable"

---

## 4. Pourquoi beaucoup de problèmes des LLM viennent de la tokenisation ?
Parce que le modèle :
- **ne voit pas des mots**,  
- mais seulement des **tokens**.

Conséquences :
- mots simples → plusieurs tokens,
- mots rares → tokenisation incohérente,
- accents, emojis, espaces → comportements étranges,
- erreurs d'interprétation → causées par la façon dont le texte est découpé.

Le modèle est limité par les choix faits lors de la création du tokenizer.

---

## 5. Pourquoi vouloir supprimer le tokenizer ?
Le tokenizer ajoute :
- rigidité,
- complexité,
- comportements bizarres,
- limitations pour certaines langues.

Idéalement, un futur LLM fonctionnerait directement :
- au niveau des **caractères**,  
ou mieux,
- au niveau des **octets**,  
sans étape de tokenisation.

Certains modèles modernes explorent déjà cette idée.


#**_____Rappel ______**

# Résumé hyper simple du fonctionnement d’un Transformeur

## 1. Tokenisation
- Le texte est découpé en unités appelées *tokens* (mots, sous-mots…).
- Chaque token est converti en un entier (ID du vocabulaire).

## 2. Embeddings + positions
- Chaque ID est transformé en un vecteur dense (*embedding*).
- On ajoute un *positional embedding* pour indiquer la position du token.

→ Résultat : une séquence de vecteurs représentant **le sens + la position** des tokens.

## 3. Self-Attention (comprendre le contexte)
Chaque token « regarde » tous les autres pour comprendre leurs relations.

Étapes internes :
1. Projection des vecteurs en Q (query), K (key), V (value).
2. Calcul des similarités entre Q et K.
3. Application d’un *softmax* pour obtenir les poids d’attention.
4. Combinaison des V selon ces poids.

→ Chaque token devient une représentation **contextuelle enrichie**.

**Multi-Head Attention** : répéter plusieurs fois l’attention en parallèle puis combiner les résultats.

## 4. Feed-Forward (raffiner l’information)
- Petit réseau : 2 couches linéaires + activation (ReLU, GELU…).
- Ajout de :
  - connexions résiduelles (*skip connections*) sortie = entrée_origine + transformation(entrée_origine)
[Améliorer les gradients: Le gradient peut “se propager” plus facilement en arrière (backpropagation), ce qui évite :
le vanishing gradient (gradient trop faible),
le exploding gradient (gradient trop fort).
Aider le réseau à apprendre plus facilement
Empêcher que le signal disparaisse:En ajoutant un chemin direct, l’information circule mieux.]
  - normalisation (*LayerNorm*).

→ Stabilise et améliore l’apprentissage.

## 5. Empilement de couches
- Les blocs “Self-Attention + Feed-Forward” sont répétés (12, 24, 96…).
- On obtient une représentation finale riche du texte.

## 6. Projection vers le vocabulaire (tête finale)

### Modèles génératifs (GPT)
- Le vecteur final de chaque token est projeté vers le vocabulaire.
- Le modèle prédit le **token suivant**.

### Modèles de classification (BERT)
- On utilise le vecteur du token spécial **[CLS]**.
- On effectue une prédiction (sentiment, entité, etc.).

## 7. Décodage (si génération)
- Conversion des logits en probabilités.
- Choix du token (greedy, sampling, beam search…).
- Ajout du token à la séquence.
- Répétition du processus jusqu’à la fin du texte.


---

# **Let's build the GPT Tokenizer**

---


Comme GPT-2, LLaMA-2 est **chunk-level**, pas character-level.

##  GPT-2 paper : 50 257 tokens, contexte 1 024 tokens
Dans *Language Models are Unsupervised Multitask Learners* (OpenAI, 2019)
- **Vocabulaire : 50 257 tokens**
- **Contexte maximal : 1 024 tokens**

[tiktokenizer ](https://tiktokenizer.vercel.app/)
## LLaMA-2 paper : même logique, mais vocabulaire différent
- vocabulaire ≈ **32k tokens** (plus petit que GPT-2)
- optimisé pour plusieurs langues
- basé sur SentencePiece (monté en BPE)



j'ai deja constater que gpt2 marche bien avec promt en anglais que d'autres langages ça revient aussi au probleme de son tokenizer

- more long large tokens in eng
- in coding handling space prob like in python

In [49]:
#we want to take strings and feed them into a language models
#how tokenize strings into some integers in some fixed vocab
#then we will use those integers to make a look up into a lookup table of vectors
#and feed those vectors into the Transformer as an input
#we want to support eng and no-english languages and specials caracteres like imogies

In [50]:
#str in python : Unicode : 149813 char
#is not stable representation
#UTF-8 ,UTF-16, UTF-32;
#UTF-8 translate to Byte stream between 1 and 4 bytes paper. UTF-8 EVERYWHERE
#ord() function

In [51]:
list("مرحبا bonjour 👋hello".encode("utf-8"))

[217,
 133,
 216,
 177,
 216,
 173,
 216,
 168,
 216,
 167,
 32,
 98,
 111,
 110,
 106,
 111,
 117,
 114,
 32,
 240,
 159,
 145,
 139,
 104,
 101,
 108,
 108,
 111]

In [52]:
list("مرحبا bonjour 👋hello".encode("utf-16"))

[255,
 254,
 69,
 6,
 49,
 6,
 45,
 6,
 40,
 6,
 39,
 6,
 32,
 0,
 98,
 0,
 111,
 0,
 110,
 0,
 106,
 0,
 111,
 0,
 117,
 0,
 114,
 0,
 32,
 0,
 61,
 216,
 75,
 220,
 104,
 0,
 101,
 0,
 108,
 0,
 108,
 0,
 111,
 0]

0.x wastefull

In [53]:
#stay with UTF-8
# atttention these are by streams vocab 256 possible tokens it's a small vocab
# all our text would be stratched out over very long sequences of bytes so the embedding table is going to be a tiny and the prediction at the top at the final layer is going to be very tiny but our sequences are very long and we have a pretty finite context length and the attention that we can support in a transformer for computational reasons and so we only have as much context length


- Avec une tokenisation byte-level pure (UTF-8), chaque token est un octet.  
→ Le vocabulaire a **256 tokens** seulement (0–255).  
- C’est un vocabulaire extrêmement petit comparé aux 30k–50k tokens de GPT ou LLaMA.
Inconvénient principal :
- **séquences de tokens très longues**
-Chaque caractère UTF-8 devient 1 à 4 bytes.  
Chaque byte devient 1 token.
-- le texte est **étiré (“stretched out”)** en énormément de tokens  
- un mot normal peut devenir 4–10 tokens  


### Contexte limité
- Les Transformers ont une **longueur de contexte finie**
- Séquences longues → modèle ne peut pas voir tout le texte
- Inefficace pour gérer de longs documents


In [54]:
 #solution # Byte Pair Encoding (BPE) Algorithm


## Idée principale BPE
- On commence avec tous les **caractères** comme tokens.
- On regarde **les paires de tokens les plus fréquentes**.
- On fusionne ces paires en un **nouveau token**.
- On répète jusqu’à atteindre la taille de vocabulaire désirée.

→ Résultat : un vocabulaire de **sous-mots optimisés** pour le texte.

Texte : `"low lower lowest"`  
BPE tokens finaux : `["low", "er", "low", "est"]`

voir article



> MEGABYTE: Predinctiong Million-byte Sequences with Multiscale Transformers




In [55]:
# text from https://www.reedbeta.com/blog/programmers-intro-to-unicode/
text = "Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception."
tokens = text.encode("utf-8") # raw bytes
tokens = list(map(int, tokens)) # convert to a list of integers in range 0..255 for convenience
print('------------------------------------')
print(text)
print("length:", len(text))
print('-------------------------------------')
print(tokens)
print("length:", len(tokens))

------------------------------------
Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception.
length: 533
-------------------------------------
[239, 188, 181, 239, 189, 142, 239, 189, 137, 239, 189, 131, 239, 189, 143, 239, 189, 132, 239, 189, 133, 33, 32, 240, 159, 133, 164, 240, 159, 133, 157, 240, 159, 133, 152, 240, 159, 133, 146, 240, 159, 133, 158, 240, 159, 133, 147, 240, 159, 133, 148, 226, 128, 189, 32, 240, 159, 135, 186, 226, 128, 140, 240, 159, 135, 179, 226, 128, 140, 240, 159, 135, 174, 226, 128, 140, 2

In [56]:
#in the more complex characters become multiple bytes
#on veux appliquer principes de BPE
#cherchant la paire that occur the most frequently
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):#crée des paires consécutives
       # Pythonic way to iterate consecutive elements
        counts[pair] = counts.get(pair, 0) + 1
    return counts

stats = get_stats(tokens)
print(stats)


{(239, 188): 1, (188, 181): 1, (181, 239): 1, (239, 189): 6, (189, 142): 1, (142, 239): 1, (189, 137): 1, (137, 239): 1, (189, 131): 1, (131, 239): 1, (189, 143): 1, (143, 239): 1, (189, 132): 1, (132, 239): 1, (189, 133): 1, (133, 33): 1, (33, 32): 2, (32, 240): 3, (240, 159): 15, (159, 133): 7, (133, 164): 1, (164, 240): 1, (133, 157): 1, (157, 240): 1, (133, 152): 1, (152, 240): 1, (133, 146): 1, (146, 240): 1, (133, 158): 1, (158, 240): 1, (133, 147): 1, (147, 240): 1, (133, 148): 1, (148, 226): 1, (226, 128): 12, (128, 189): 1, (189, 32): 1, (159, 135): 7, (135, 186): 1, (186, 226): 1, (128, 140): 6, (140, 240): 6, (135, 179): 1, (179, 226): 1, (135, 174): 1, (174, 226): 1, (135, 168): 1, (168, 226): 1, (135, 180): 1, (180, 226): 1, (135, 169): 1, (169, 226): 1, (135, 170): 1, (170, 33): 1, (159, 152): 1, (152, 132): 1, (132, 32): 1, (32, 84): 1, (84, 104): 1, (104, 101): 6, (101, 32): 20, (32, 118): 1, (118, 101): 3, (101, 114): 6, (114, 121): 2, (121, 32): 2, (32, 110): 2, (110,

In [57]:
print(sorted(((v,k) for k,v in stats.items()), reverse=True))


[(20, (101, 32)), (15, (240, 159)), (12, (226, 128)), (12, (105, 110)), (10, (115, 32)), (10, (97, 110)), (10, (32, 97)), (9, (32, 116)), (8, (116, 104)), (7, (159, 135)), (7, (159, 133)), (7, (97, 114)), (6, (239, 189)), (6, (140, 240)), (6, (128, 140)), (6, (116, 32)), (6, (114, 32)), (6, (111, 114)), (6, (110, 103)), (6, (110, 100)), (6, (109, 101)), (6, (104, 101)), (6, (101, 114)), (6, (32, 105)), (5, (117, 115)), (5, (115, 116)), (5, (110, 32)), (5, (100, 101)), (5, (44, 32)), (5, (32, 115)), (4, (116, 105)), (4, (116, 101)), (4, (115, 44)), (4, (114, 105)), (4, (111, 117)), (4, (111, 100)), (4, (110, 116)), (4, (110, 105)), (4, (105, 99)), (4, (104, 97)), (4, (103, 32)), (4, (101, 97)), (4, (100, 32)), (4, (99, 111)), (4, (97, 109)), (4, (85, 110)), (4, (32, 119)), (4, (32, 111)), (4, (32, 102)), (4, (32, 85)), (3, (118, 101)), (3, (116, 115)), (3, (116, 114)), (3, (116, 111)), (3, (114, 116)), (3, (114, 115)), (3, (114, 101)), (3, (111, 102)), (3, (111, 32)), (3, (108, 108)), (

char()  , ord()

In [58]:
# after menting a new token with the ID of 256 right because max actuelle est 255
#(101, 32) = 256
top_pair = max(stats, key=stats.get)
top_pair

(101, 32)

In [59]:
# on a creer une fonction merge par ce que a chaqye iteration on va merger
def merge(ids, pair, idx):
  # in the list of ints (ids), replace all consecutive occurences of pair with the new token idx
  newids = []
  i = 0
  while i < len(ids):
    # if we are not at the very last position AND the pair matches, replace it
    if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
      newids.append(idx)
      i += 2
    else:
      newids.append(ids[i])
      i += 1
  return newids

print(merge([5, 6, 6, 7, 9, 1], (6, 7), 99))


[5, 6, 99, 9, 1]


In [60]:
tokens2 = merge(tokens, top_pair, 256)
print(tokens2)
print("length:", len(tokens2))

[239, 188, 181, 239, 189, 142, 239, 189, 137, 239, 189, 131, 239, 189, 143, 239, 189, 132, 239, 189, 133, 33, 32, 240, 159, 133, 164, 240, 159, 133, 157, 240, 159, 133, 152, 240, 159, 133, 146, 240, 159, 133, 158, 240, 159, 133, 147, 240, 159, 133, 148, 226, 128, 189, 32, 240, 159, 135, 186, 226, 128, 140, 240, 159, 135, 179, 226, 128, 140, 240, 159, 135, 174, 226, 128, 140, 240, 159, 135, 168, 226, 128, 140, 240, 159, 135, 180, 226, 128, 140, 240, 159, 135, 169, 226, 128, 140, 240, 159, 135, 170, 33, 32, 240, 159, 152, 132, 32, 84, 104, 256, 118, 101, 114, 121, 32, 110, 97, 109, 256, 115, 116, 114, 105, 107, 101, 115, 32, 102, 101, 97, 114, 32, 97, 110, 100, 32, 97, 119, 256, 105, 110, 116, 111, 32, 116, 104, 256, 104, 101, 97, 114, 116, 115, 32, 111, 102, 32, 112, 114, 111, 103, 114, 97, 109, 109, 101, 114, 115, 32, 119, 111, 114, 108, 100, 119, 105, 100, 101, 46, 32, 87, 256, 97, 108, 108, 32, 107, 110, 111, 119, 32, 119, 256, 111, 117, 103, 104, 116, 32, 116, 111, 32, 226, 128, 156

In [61]:
# de 616 tokens a 596
stats = get_stats(tokens2)
top_pair = max(stats, key=stats.get)
top_pair

(240, 159)

In [62]:
#la question how many times on repete le processus
# heee bas voila un hyperparameter
# the more steps we take the larger will be our vocab genre 256 257 ....
# and the shorter will be our sequence # de 616 tokens a 596
#so it's a hyperparameter we tune it and we find goog vocab sizes

In [63]:
#travaillons sur text plus grand
# making the training text longer to have more representative token statistics
# text from https://www.reedbeta.com/blog/programmers-intro-to-unicode/
text = """A Programmer’s Introduction to Unicode March 3, 2017 · Coding · 22 Comments  Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺\u200c🇳\u200c🇮\u200c🇨\u200c🇴\u200c🇩\u200c🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception.  A few months ago, I got interested in Unicode and decided to spend some time learning more about it in detail. In this article, I’ll give an introduction to it from a programmer’s point of view.  I’m going to focus on the character set and what’s involved in working with strings and files of Unicode text. However, in this article I’m not going to talk about fonts, text layout/shaping/rendering, or localization in detail—those are separate issues, beyond my scope (and knowledge) here.  Diversity and Inherent Complexity The Unicode Codespace Codespace Allocation Scripts Usage Frequency Encodings UTF-8 UTF-16 Combining Marks Canonical Equivalence Normalization Forms Grapheme Clusters And More… Diversity and Inherent Complexity As soon as you start to study Unicode, it becomes clear that it represents a large jump in complexity over character sets like ASCII that you may be more familiar with. It’s not just that Unicode contains a much larger number of characters, although that’s part of it. Unicode also has a great deal of internal structure, features, and special cases, making it much more than what one might expect a mere “character set” to be. We’ll see some of that later in this article.  When confronting all this complexity, especially as an engineer, it’s hard not to find oneself asking, “Why do we need all this? Is this really necessary? Couldn’t it be simplified?”  However, Unicode aims to faithfully represent the entire world’s writing systems. The Unicode Consortium’s stated goal is “enabling people around the world to use computers in any language”. And as you might imagine, the diversity of written languages is immense! To date, Unicode supports 135 different scripts, covering some 1100 languages, and there’s still a long tail of over 100 unsupported scripts, both modern and historical, which people are still working to add.  Given this enormous diversity, it’s inevitable that representing it is a complicated project. Unicode embraces that diversity, and accepts the complexity inherent in its mission to include all human writing systems. It doesn’t make a lot of trade-offs in the name of simplification, and it makes exceptions to its own rules where necessary to further its mission.  Moreover, Unicode is committed not just to supporting texts in any single language, but also to letting multiple languages coexist within one text—which introduces even more complexity.  Most programming languages have libraries available to handle the gory low-level details of text manipulation, but as a programmer, you’ll still need to know about certain Unicode features in order to know when and how to apply them. It may take some time to wrap your head around it all, but don’t be discouraged—think about the billions of people for whom your software will be more accessible through supporting text in their language. Embrace the complexity!  The Unicode Codespace Let’s start with some general orientation. The basic elements of Unicode—its “characters”, although that term isn’t quite right—are called code points. Code points are identified by number, customarily written in hexadecimal with the prefix “U+”, such as U+0041 “A” latin capital letter a or U+03B8 “θ” greek small letter theta. Each code point also has a short name, and quite a few other properties, specified in the Unicode Character Database.  The set of all possible code points is called the codespace. The Unicode codespace consists of 1,114,112 code points. However, only 128,237 of them—about 12% of the codespace—are actually assigned, to date. There’s plenty of room for growth! Unicode also reserves an additional 137,468 code points as “private use” areas, which have no standardized meaning and are available for individual applications to define for their own purposes.  Codespace Allocation To get a feel for how the codespace is laid out, it’s helpful to visualize it. Below is a map of the entire codespace, with one pixel per code point. It’s arranged in tiles for visual coherence; each small square is 16×16 = 256 code points, and each large square is a “plane” of 65,536 code points. There are 17 planes altogether.  Map of the Unicode codespace (click to zoom)  White represents unassigned space. Blue is assigned code points, green is private-use areas, and the small red area is surrogates (more about those later). As you can see, the assigned code points are distributed somewhat sparsely, but concentrated in the first three planes.  Plane 0 is also known as the “Basic Multilingual Plane”, or BMP. The BMP contains essentially all the characters needed for modern text in any script, including Latin, Cyrillic, Greek, Han (Chinese), Japanese, Korean, Arabic, Hebrew, Devanagari (Indian), and many more.  (In the past, the codespace was just the BMP and no more—Unicode was originally conceived as a straightforward 16-bit encoding, with only 65,536 code points. It was expanded to its current size in 1996. However, the vast majority of code points in modern text belong to the BMP.)  Plane 1 contains historical scripts, such as Sumerian cuneiform and Egyptian hieroglyphs, as well as emoji and various other symbols. Plane 2 contains a large block of less-common and historical Han characters. The remaining planes are empty, except for a small number of rarely-used formatting characters in Plane 14; planes 15–16 are reserved entirely for private use.  Scripts Let’s zoom in on the first three planes, since that’s where the action is:  Map of scripts in Unicode planes 0–2 (click to zoom)  This map color-codes the 135 different scripts in Unicode. You can see how Han () and Korean () take up most of the range of the BMP (the left large square). By contrast, all of the European, Middle Eastern, and South Asian scripts fit into the first row of the BMP in this diagram.  Many areas of the codespace are adapted or copied from earlier encodings. For example, the first 128 code points of Unicode are just a copy of ASCII. This has clear benefits for compatibility—it’s easy to losslessly convert texts from smaller encodings into Unicode (and the other direction too, as long as no characters outside the smaller encoding are used).  Usage Frequency One more interesting way to visualize the codespace is to look at the distribution of usage—in other words, how often each code point is actually used in real-world texts. Below is a heat map of planes 0–2 based on a large sample of text from Wikipedia and Twitter (all languages). Frequency increases from black (never seen) through red and yellow to white.  Heat map of code point usage frequency in Unicode planes 0–2 (click to zoom)  You can see that the vast majority of this text sample lies in the BMP, with only scattered usage of code points from planes 1–2. The biggest exception is emoji, which show up here as the several bright squares in the bottom row of plane 1.  Encodings We’ve seen that Unicode code points are abstractly identified by their index in the codespace, ranging from U+0000 to U+10FFFF. But how do code points get represented as bytes, in memory or in a file?  The most convenient, computer-friendliest (and programmer-friendliest) thing to do would be to just store the code point index as a 32-bit integer. This works, but it consumes 4 bytes per code point, which is sort of a lot. Using 32-bit ints for Unicode will cost you a bunch of extra storage, memory, and performance in bandwidth-bound scenarios, if you work with a lot of text.  Consequently, there are several more-compact encodings for Unicode. The 32-bit integer encoding is officially called UTF-32 (UTF = “Unicode Transformation Format”), but it’s rarely used for storage. At most, it comes up sometimes as a temporary internal representation, for examining or operating on the code points in a string.  Much more commonly, you’ll see Unicode text encoded as either UTF-8 or UTF-16. These are both variable-length encodings, made up of 8-bit or 16-bit units, respectively. In these schemes, code points with smaller index values take up fewer bytes, which saves a lot of memory for typical texts. The trade-off is that processing UTF-8/16 texts is more programmatically involved, and likely slower.  UTF-8 In UTF-8, each code point is stored using 1 to 4 bytes, based on its index value.  UTF-8 uses a system of binary prefixes, in which the high bits of each byte mark whether it’s a single byte, the beginning of a multi-byte sequence, or a continuation byte; the remaining bits, concatenated, give the code point index. This table shows how it works:  UTF-8 (binary)\tCode point (binary)\tRange 0xxxxxxx\txxxxxxx\tU+0000–U+007F 110xxxxx 10yyyyyy\txxxxxyyyyyy\tU+0080–U+07FF 1110xxxx 10yyyyyy 10zzzzzz\txxxxyyyyyyzzzzzz\tU+0800–U+FFFF 11110xxx 10yyyyyy 10zzzzzz 10wwwwww\txxxyyyyyyzzzzzzwwwwww\tU+10000–U+10FFFF A handy property of UTF-8 is that code points below 128 (ASCII characters) are encoded as single bytes, and all non-ASCII code points are encoded using sequences of bytes 128–255. This has a couple of nice consequences. First, any strings or files out there that are already in ASCII can also be interpreted as UTF-8 without any conversion. Second, lots of widely-used string programming idioms—such as null termination, or delimiters (newlines, tabs, commas, slashes, etc.)—will just work on UTF-8 strings. ASCII bytes never occur inside the encoding of non-ASCII code points, so searching byte-wise for a null terminator or a delimiter will do the right thing.  Thanks to this convenience, it’s relatively simple to extend legacy ASCII programs and APIs to handle UTF-8 strings. UTF-8 is very widely used in the Unix/Linux and Web worlds, and many programmers argue UTF-8 should be the default encoding everywhere.  However, UTF-8 isn’t a drop-in replacement for ASCII strings in all respects. For instance, code that iterates over the “characters” in a string will need to decode UTF-8 and iterate over code points (or maybe grapheme clusters—more about those later), not bytes. When you measure the “length” of a string, you’ll need to think about whether you want the length in bytes, the length in code points, the width of the text when rendered, or something else.  UTF-16 The other encoding that you’re likely to encounter is UTF-16. It uses 16-bit words, with each code point stored as either 1 or 2 words.  Like UTF-8, we can express the UTF-16 encoding rules in the form of binary prefixes:  UTF-16 (binary)\tCode point (binary)\tRange xxxxxxxxxxxxxxxx\txxxxxxxxxxxxxxxx\tU+0000–U+FFFF 110110xxxxxxxxxx 110111yyyyyyyyyy\txxxxxxxxxxyyyyyyyyyy + 0x10000\tU+10000–U+10FFFF A more common way that people talk about UTF-16 encoding, though, is in terms of code points called “surrogates”. All the code points in the range U+D800–U+DFFF—or in other words, the code points that match the binary prefixes 110110 and 110111 in the table above—are reserved specifically for UTF-16 encoding, and don’t represent any valid characters on their own. They’re only meant to occur in the 2-word encoding pattern above, which is called a “surrogate pair”. Surrogate code points are illegal in any other context! They’re not allowed in UTF-8 or UTF-32 at all.  Historically, UTF-16 is a descendant of the original, pre-1996 versions of Unicode, in which there were only 65,536 code points. The original intention was that there would be no different “encodings”; Unicode was supposed to be a straightforward 16-bit character set. Later, the codespace was expanded to make room for a long tail of less-common (but still important) Han characters, which the Unicode designers didn’t originally plan for. Surrogates were then introduced, as—to put it bluntly—a kludge, allowing 16-bit encodings to access the new code points.  Today, Javascript uses UTF-16 as its standard string representation: if you ask for the length of a string, or iterate over it, etc., the result will be in UTF-16 words, with any code points outside the BMP expressed as surrogate pairs. UTF-16 is also used by the Microsoft Win32 APIs; though Win32 supports either 8-bit or 16-bit strings, the 8-bit version unaccountably still doesn’t support UTF-8—only legacy code-page encodings, like ANSI. This leaves UTF-16 as the only way to get proper Unicode support in Windows. (Update: in Win10 version 1903, they finally added UTF-8 support to the 8-bit APIs! 😊)  By the way, UTF-16’s words can be stored either little-endian or big-endian. Unicode has no opinion on that issue, though it does encourage the convention of putting U+FEFF zero width no-break space at the top of a UTF-16 file as a byte-order mark, to disambiguate the endianness. (If the file doesn’t match the system’s endianness, the BOM will be decoded as U+FFFE, which isn’t a valid code point.)  Combining Marks In the story so far, we’ve been focusing on code points. But in Unicode, a “character” can be more complicated than just an individual code point!  Unicode includes a system for dynamically composing characters, by combining multiple code points together. This is used in various ways to gain flexibility without causing a huge combinatorial explosion in the number of code points.  In European languages, for example, this shows up in the application of diacritics to letters. Unicode supports a wide range of diacritics, including acute and grave accents, umlauts, cedillas, and many more. All these diacritics can be applied to any letter of any alphabet—and in fact, multiple diacritics can be used on a single letter.  If Unicode tried to assign a distinct code point to every possible combination of letter and diacritics, things would rapidly get out of hand. Instead, the dynamic composition system enables you to construct the character you want, by starting with a base code point (the letter) and appending additional code points, called “combining marks”, to specify the diacritics. When a text renderer sees a sequence like this in a string, it automatically stacks the diacritics over or under the base letter to create a composed character.  For example, the accented character “Á” can be expressed as a string of two code points: U+0041 “A” latin capital letter a plus U+0301 “◌́” combining acute accent. This string automatically gets rendered as a single character: “Á”.  Now, Unicode does also include many “precomposed” code points, each representing a letter with some combination of diacritics already applied, such as U+00C1 “Á” latin capital letter a with acute or U+1EC7 “ệ” latin small letter e with circumflex and dot below. I suspect these are mostly inherited from older encodings that were assimilated into Unicode, and kept around for compatibility. In practice, there are precomposed code points for most of the common letter-with-diacritic combinations in European-script languages, so they don’t use dynamic composition that much in typical text.  Still, the system of combining marks does allow for an arbitrary number of diacritics to be stacked on any base character. The reductio-ad-absurdum of this is Zalgo text, which works by ͖͟ͅr͞aṋ̫̠̖͈̗d͖̻̹óm̪͙͕̗̝ļ͇̰͓̳̫ý͓̥̟͍ ̕s̫t̫̱͕̗̰̼̘͜a̼̩͖͇̠͈̣͝c̙͍k̖̱̹͍͘i̢n̨̺̝͇͇̟͙ģ̫̮͎̻̟ͅ ̕n̼̺͈͞u̮͙m̺̭̟̗͞e̞͓̰̤͓̫r̵o̖ṷs҉̪͍̭̬̝̤ ̮͉̝̞̗̟͠d̴̟̜̱͕͚i͇̫̼̯̭̜͡ḁ͙̻̼c̲̲̹r̨̠̹̣̰̦i̱t̤̻̤͍͙̘̕i̵̜̭̤̱͎c̵s ͘o̱̲͈̙͖͇̲͢n͘ ̜͈e̬̲̠̩ac͕̺̠͉h̷̪ ̺̣͖̱ḻ̫̬̝̹ḙ̙̺͙̭͓̲t̞̞͇̲͉͍t̷͔̪͉̲̻̠͙e̦̻͈͉͇r͇̭̭̬͖,̖́ ̜͙͓̣̭s̘̘͈o̱̰̤̲ͅ ̛̬̜̙t̼̦͕̱̹͕̥h̳̲͈͝ͅa̦t̻̲ ̻̟̭̦̖t̛̰̩h̠͕̳̝̫͕e͈̤̘͖̞͘y҉̝͙ ̷͉͔̰̠o̞̰v͈͈̳̘͜er̶f̰͈͔ḻ͕̘̫̺̲o̲̭͙͠ͅw̱̳̺ ͜t̸h͇̭͕̳͍e̖̯̟̠ ͍̞̜͔̩̪͜ļ͎̪̲͚i̝̲̹̙̩̹n̨̦̩̖ḙ̼̲̼͢ͅ ̬͝s̼͚̘̞͝p͙̘̻a̙c҉͉̜̤͈̯̖i̥͡n̦̠̱͟g̸̗̻̦̭̮̟ͅ ̳̪̠͖̳̯̕a̫͜n͝d͡ ̣̦̙ͅc̪̗r̴͙̮̦̹̳e͇͚̞͔̹̫͟a̙̺̙ț͔͎̘̹ͅe̥̩͍ a͖̪̜̮͙̹n̢͉̝ ͇͉͓̦̼́a̳͖̪̤̱p̖͔͔̟͇͎͠p̱͍̺ę̲͎͈̰̲̤̫a̯͜r̨̮̫̣̘a̩̯͖n̹̦̰͎̣̞̞c̨̦̱͔͎͍͖e̬͓͘ ̤̰̩͙̤̬͙o̵̼̻̬̻͇̮̪f̴ ̡̙̭͓͖̪̤“̸͙̠̼c̳̗͜o͏̼͙͔̮r̞̫̺̞̥̬ru̺̻̯͉̭̻̯p̰̥͓̣̫̙̤͢t̳͍̳̖ͅi̶͈̝͙̼̙̹o̡͔n̙̺̹̖̩͝ͅ”̨̗͖͚̩.̯͓  A few other places where dynamic character composition shows up in Unicode:  Vowel-pointing notation in Arabic and Hebrew. In these languages, words are normally spelled with some of their vowels left out. They then have diacritic notation to indicate the vowels (used in dictionaries, language-teaching materials, children’s books, and such). These diacritics are expressed with combining marks.  A Hebrew example, with niqqud:\tאֶת דַלְתִּי הֵזִיז הֵנִיעַ, קֶטֶב לִשְׁכַּתִּי יָשׁוֹד Normal writing (no niqqud):\tאת דלתי הזיז הניע, קטב לשכתי ישוד Devanagari, the script used to write Hindi, Sanskrit, and many other South Asian languages, expresses certain vowels as combining marks attached to consonant letters. For example, “ह” + “\u200bि” = “हि” (“h” + “i” = “hi”). Korean characters stand for syllables, but they are composed of letters called jamo that stand for the vowels and consonants in the syllable. While there are code points for precomposed Korean syllables, it’s also possible to dynamically compose them by concatenating their jamo. For example, “ᄒ” + “ᅡ” + “ᆫ” = “한” (“h” + “a” + “n” = “han”). Canonical Equivalence In Unicode, precomposed characters exist alongside the dynamic composition system. A consequence of this is that there are multiple ways to express “the same” string—different sequences of code points that result in the same user-perceived characters. For example, as we saw earlier, we can express the character “Á” either as the single code point U+00C1, or as the string of two code points U+0041 U+0301.  Another source of ambiguity is the ordering of multiple diacritics in a single character. Diacritic order matters visually when two diacritics apply to the same side of the base character, e.g. both above: “ǡ” (dot, then macron) is different from “ā̇” (macron, then dot). However, when diacritics apply to different sides of the character, e.g. one above and one below, then the order doesn’t affect rendering. Moreover, a character with multiple diacritics might have one of the diacritics precomposed and others expressed as combining marks.  For example, the Vietnamese letter “ệ” can be expressed in five different ways:  Fully precomposed: U+1EC7 “ệ” Partially precomposed: U+1EB9 “ẹ” + U+0302 “◌̂” Partially precomposed: U+00EA “ê” + U+0323 “◌̣” Fully decomposed: U+0065 “e” + U+0323 “◌̣” + U+0302 “◌̂” Fully decomposed: U+0065 “e” + U+0302 “◌̂” + U+0323 “◌̣” Unicode refers to set of strings like this as “canonically equivalent”. Canonically equivalent strings are supposed to be treated as identical for purposes of searching, sorting, rendering, text selection, and so on. This has implications for how you implement operations on text. For example, if an app has a “find in file” operation and the user searches for “ệ”, it should, by default, find occurrences of any of the five versions of “ệ” above!  Normalization Forms To address the problem of “how to handle canonically equivalent strings”, Unicode defines several normalization forms: ways of converting strings into a canonical form so that they can be compared code-point-by-code-point (or byte-by-byte).  The “NFD” normalization form fully decomposes every character down to its component base and combining marks, taking apart any precomposed code points in the string. It also sorts the combining marks in each character according to their rendered position, so e.g. diacritics that go below the character come before the ones that go above the character. (It doesn’t reorder diacritics in the same rendered position, since their order matters visually, as previously mentioned.)  The “NFC” form, conversely, puts things back together into precomposed code points as much as possible. If an unusual combination of diacritics is called for, there may not be any precomposed code point for it, in which case NFC still precomposes what it can and leaves any remaining combining marks in place (again ordered by rendered position, as in NFD).  There are also forms called NFKD and NFKC. The “K” here refers to compatibility decompositions, which cover characters that are “similar” in some sense but not visually identical. However, I’m not going to cover that here.  Grapheme Clusters As we’ve seen, Unicode contains various cases where a thing that a user thinks of as a single “character” might actually be made up of multiple code points under the hood. Unicode formalizes this using the notion of a grapheme cluster: a string of one or more code points that constitute a single “user-perceived character”.  UAX #29 defines the rules for what, precisely, qualifies as a grapheme cluster. It’s approximately “a base code point followed by any number of combining marks”, but the actual definition is a bit more complicated; it accounts for things like Korean jamo, and emoji ZWJ sequences.  The main thing grapheme clusters are used for is text editing: they’re often the most sensible unit for cursor placement and text selection boundaries. Using grapheme clusters for these purposes ensures that you can’t accidentally chop off some diacritics when you copy-and-paste text, that left/right arrow keys always move the cursor by one visible character, and so on.  Another place where grapheme clusters are useful is in enforcing a string length limit—say, on a database field. While the true, underlying limit might be something like the byte length of the string in UTF-8, you wouldn’t want to enforce that by just truncating bytes. At a minimum, you’d want to “round down” to the nearest code point boundary; but even better, round down to the nearest grapheme cluster boundary. Otherwise, you might be corrupting the last character by cutting off a diacritic, or interrupting a jamo sequence or ZWJ sequence.  And More… There’s much more that could be said about Unicode from a programmer’s perspective! I haven’t gotten into such fun topics as case mapping, collation, compatibility decompositions and confusables, Unicode-aware regexes, or bidirectional text. Nor have I said anything yet about implementation issues—how to efficiently store and look-up data about the sparsely-assigned code points, or how to optimize UTF-8 decoding, string comparison, or NFC normalization. Perhaps I’ll return to some of those things in future posts.  Unicode is a fascinating and complex system. It has a many-to-one mapping between bytes and code points, and on top of that a many-to-one (or, under some circumstances, many-to-many) mapping between code points and “characters”. It has oddball special cases in every corner. But no one ever claimed that representing all written languages was going to be easy, and it’s clear that we’re never going back to the bad old days of a patchwork of incompatible encodings.  Further reading:  The Unicode Standard UTF-8 Everywhere Manifesto Dark corners of Unicode by Eevee ICU (International Components for Unicode)—C/C++/Java libraries implementing many Unicode algorithms and related things Python 3 Unicode Howto Google Noto Fonts—set of fonts intended to cover all assigned code points"""
tokens = text.encode("utf-8") # raw bytes
tokens = list(map(int, tokens)) # convert to a list of integers in range 0..255 for convenience

In [64]:
#t'inquiete j'ai juste copier the same functions
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids, pair, idx):
  newids = []
  i = 0
  while i < len(ids):
    if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
      newids.append(idx)
      i += 2
    else:
      newids.append(ids[i])
      i += 1
  return newids



In [65]:
# --- essayons 276 = 20 merges
vocab_size = 276 # the desired final vocabulary size
num_merges = vocab_size - 256
ids = list(tokens) # copy so we don't destroy the original list

merges = {} # (int, int) -> int like #(101, 32) =  idx256
for i in range(num_merges):
  stats = get_stats(ids)
  pair = max(stats, key=stats.get)
  idx = 256 + i
  print(f"merging {pair} into a new token {idx}")
  ids = merge(ids, pair, idx)
  merges[pair] = idx

merging (101, 32) into a new token 256
merging (105, 110) into a new token 257
merging (115, 32) into a new token 258
merging (116, 104) into a new token 259
merging (101, 114) into a new token 260
merging (99, 111) into a new token 261
merging (116, 32) into a new token 262
merging (226, 128) into a new token 263
merging (44, 32) into a new token 264
merging (97, 110) into a new token 265
merging (111, 114) into a new token 266
merging (100, 32) into a new token 267
merging (97, 114) into a new token 268
merging (101, 110) into a new token 269
merging (257, 103) into a new token 270
merging (261, 100) into a new token 271
merging (121, 32) into a new token 272
merging (46, 32) into a new token 273
merging (97, 108) into a new token 274
merging (259, 256) into a new token 275


In [66]:
print("tokens length:", len(tokens))
print("ids length:", len(ids))
print(f"compression ratio: {len(tokens) / len(ids):.2f}X")

tokens length: 24597
ids length: 19438
compression ratio: 1.27X


rappelons que tokenizer est totalement séparé du LLM it's a separate object it has a different training set
it is a pre processing stage that we would run a single time in the beginning
and the tokenizer use BPE

once it's trained : we have the vocab + merges = we can do encoding and decoding

*so it's like a translation layer*

In [67]:
#text -> Tokenizer -> tokens
# tokens -> Tokenizer -> text

# we care about multi many different languages and is it a code or not
# the training set determine how many merges of it there will be

## **encoding + decoding**
text -> Tokenizer -> tokens

tokens -> Tokenizer -> text



**decoding**

In [68]:
vocab = {idx: bytes([idx]) for idx in range(256)}
#vocabulaire de bytes pour tous les octets possibles (0–255).
for (p0, p1), idx in merges.items():
  #merges = dictionnaire qui contient les fusions BPE : (20 items) qu'on a merger {(101,32):256,....}
  #clé = paire de tokens (p0, p1) and idx val
    vocab[idx] = vocab[p0] + vocab[p1]
  #On construit le nouveau token fusionné en concaténant les bytes existants

#vocab contient les tokens initiaux + tous les sous-mots fusionnés.
def decode(ids):
  # given ids (list of integers), return Python string
  tokens = b"".join(vocab[idx] for idx in ids)
  text = tokens.decode("utf-8", errors="replace")
  return text

print(decode([128])) #�
#decode [97] a but , [128] error can't decode 0x80 in position0
#sol : use  , errors="replace"

�


In [69]:
#vocab contient les tokens initiaux + tous les sous-mots fusionnés.
print(decode([259]))

th


**encoding**

In [70]:
merges

{(101, 32): 256,
 (105, 110): 257,
 (115, 32): 258,
 (116, 104): 259,
 (101, 114): 260,
 (99, 111): 261,
 (116, 32): 262,
 (226, 128): 263,
 (44, 32): 264,
 (97, 110): 265,
 (111, 114): 266,
 (100, 32): 267,
 (97, 114): 268,
 (101, 110): 269,
 (257, 103): 270,
 (261, 100): 271,
 (121, 32): 272,
 (46, 32): 273,
 (97, 108): 274,
 (259, 256): 275}

In [71]:
#comment on va encoder on peut utiliser au debut encodage utf 8 mais
#on doit utiliser aussi les merges mais comment y a un ordre a respecter genre
# comme de aaabdaaabac -> ZabdZabac -> ZYdZYac -> XdXac
# chercher (101, 32)  ..... avant  (259, 256)
#respecter ordre est simple voir the key 257<259


In [72]:
def encode(text):
  # given a string, return list of integers (the tokens)
  tokens = list(text.encode("utf-8"))
  while len(tokens) >= 2: #eviter erreur encode("a")
    stats = get_stats(tokens)
    #respecter ordre est simple voir the key 257<259
    pair = min(stats, key=lambda p: merges.get(p, float("inf")))
    if pair not in merges:
      break # nothing else can be merged
    idx = merges[pair]
    tokens = merge(tokens, pair, idx)
  return tokens

print(encode("hi it's a"))

[104, 105, 32, 105, 116, 39, 258, 97]


In [73]:
print(decode(encode("hi it's a")))

hi it's a


In [74]:
valtext =  "On a $50,000 mortgage of 30 years at 8 percent, the monthly payment would be $366.88."
valtext2 = decode(encode(valtext))
print(valtext2 == valtext)

True


---

# Forced splits using regex patterns (GPT series)

> Language Models are Unsupervised Multitask Learners

dog. dog dog! it's like semantic+ pontuation = suboptimal
some type of caracters should never be merged together



> openai/gpt-2

```
# Should have added re.IGNORECASE so BPE merges can happen for capitalized versions of contractions

self.pat = re.compile(r"''|``|\'|\"|m|ll|d|re|s|ve|t|em|m'\w+|['\p{L}]+|…|[A-Za-z]+|(\w+)|...?")
```





In [75]:
import regex as re
#lowercase
gpt2pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")
#découper le texte en tokens avant d’appliquer BPE
print(re.findall(gpt2pat, "Hello've world123 how's are you!!!?"))

['Hello', "'ve", ' world', '123', ' how', "'s", ' are', ' you', '!!!?']


L’expression découpe le texte en tokens adaptés à GPT-2 :

Contractions anglaises → 's, 're

Mots → lettres avec espace optionnel

Nombres → chiffres avec espace optionnel

Symboles/punctuations → avec espace optionnel

Espaces isolés → préservés

In [76]:
example = """
for i in range(1, 101):
    if i % 3 == 0 and i % 5 == 0:
        print("FizzBuzz")
    elif i % 3 == 0:
        print("Fizz")
    elif i % 5 == 0:
        print("Buzz")
    else:
        print(i)
"""
print(re.findall(gpt2pat, example))

['\n', 'for', ' i', ' in', ' range', '(', '1', ',', ' 101', '):', '\n   ', ' if', ' i', ' %', ' 3', ' ==', ' 0', ' and', ' i', ' %', ' 5', ' ==', ' 0', ':', '\n       ', ' print', '("', 'FizzBuzz', '")', '\n   ', ' elif', ' i', ' %', ' 3', ' ==', ' 0', ':', '\n       ', ' print', '("', 'Fizz', '")', '\n   ', ' elif', ' i', ' %', ' 5', ' ==', ' 0', ':', '\n       ', ' print', '("', 'Buzz', '")', '\n   ', ' else', ':', '\n       ', ' print', '(', 'i', ')', '\n']


In [77]:
#white space is included at the beginning, but if there are multiple spaces, it preserves them.

In [78]:
!pip install tiktoken # added for colab

In [79]:
# in gpt4 they change the regular expression that they use to chunk up the text
# not more than 3 numbers merging
# upper and lower
# on va appliquer leurs algo sur notre vocab et merges
# ils ont byte_encoder byte_decoder layers entre Tokenizer et raw_text

In [80]:
import tiktoken

# GPT-2 (does not merge spaces)
enc = tiktoken.get_encoding("gpt2")
print(enc.encode("    hello world!!!"))

# GPT-4 (merges spaces)
enc = tiktoken.get_encoding("cl100k_base")
print(enc.encode("    hello world!!!"))

[220, 220, 220, 23748, 995, 10185]
[262, 24748, 1917, 12340]


In [81]:
!wget https://openaipublic.blob.core.windows.net/gpt-2/models/1558M/vocab.bpe
!wget https://openaipublic.blob.core.windows.net/gpt-2/models/1558M/encoder.json

--2025-12-08 20:49:43--  https://openaipublic.blob.core.windows.net/gpt-2/models/1558M/vocab.bpe
Resolving openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)... 20.60.241.33
Connecting to openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)|20.60.241.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 456318 (446K) [application/octet-stream]
Saving to: ‘vocab.bpe.1’

vocab.bpe.1         100%[===================>] 445.62K  1.63MB/s    in 0.3s    

2025-12-08 20:49:44 (1.63 MB/s) - ‘vocab.bpe.1’ saved [456318/456318]

--2025-12-08 20:49:44--  https://openaipublic.blob.core.windows.net/gpt-2/models/1558M/encoder.json
Resolving openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)... 20.60.241.33
Connecting to openaipublic.blob.core.windows.net (openaipublic.blob.core.windows.net)|20.60.241.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1042301 (1018K) [application/json]
Saving 

In [82]:
import os, json

with open('encoder.json', 'r') as f:
    encoder = json.load(f) # <--- ~equivalent to our "vocab"

with open('vocab.bpe', 'r', encoding="utf-8") as f:
    bpe_data = f.read()
bpe_merges = [tuple(merge_str.split()) for merge_str in bpe_data.split('\n')[1:-1]]
# ^---- ~equivalent to our "merges"


In [83]:
#encoder.json contains a mapping of tokens → integer IDs
#n GPT/BPE terminology, this is the merges dictionary.

##Each pair will be merged iteratively during tokenization.

### bpr cherche les bigramm donc les paires et merge selon la freq

**special tokens**

In [84]:
print(len(encoder) )
# 256 raw byte tokens.
#50,000 merges.
#+1 special token
encoder['<|endoftext|>']

50257


50256

In [85]:
encoder

{'!': 0,
 '"': 1,
 '#': 2,
 '$': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 '(': 7,
 ')': 8,
 '*': 9,
 '+': 10,
 ',': 11,
 '-': 12,
 '.': 13,
 '/': 14,
 '0': 15,
 '1': 16,
 '2': 17,
 '3': 18,
 '4': 19,
 '5': 20,
 '6': 21,
 '7': 22,
 '8': 23,
 '9': 24,
 ':': 25,
 ';': 26,
 '<': 27,
 '=': 28,
 '>': 29,
 '?': 30,
 '@': 31,
 'A': 32,
 'B': 33,
 'C': 34,
 'D': 35,
 'E': 36,
 'F': 37,
 'G': 38,
 'H': 39,
 'I': 40,
 'J': 41,
 'K': 42,
 'L': 43,
 'M': 44,
 'N': 45,
 'O': 46,
 'P': 47,
 'Q': 48,
 'R': 49,
 'S': 50,
 'T': 51,
 'U': 52,
 'V': 53,
 'W': 54,
 'X': 55,
 'Y': 56,
 'Z': 57,
 '[': 58,
 '\\': 59,
 ']': 60,
 '^': 61,
 '_': 62,
 '`': 63,
 'a': 64,
 'b': 65,
 'c': 66,
 'd': 67,
 'e': 68,
 'f': 69,
 'g': 70,
 'h': 71,
 'i': 72,
 'j': 73,
 'k': 74,
 'l': 75,
 'm': 76,
 'n': 77,
 'o': 78,
 'p': 79,
 'q': 80,
 'r': 81,
 's': 82,
 't': 83,
 'u': 84,
 'v': 85,
 'w': 86,
 'x': 87,
 'y': 88,
 'z': 89,
 '{': 90,
 '|': 91,
 '}': 92,
 '~': 93,
 '¡': 94,
 '¢': 95,
 '£': 96,
 '¤': 97,
 '¥': 98,
 '¦': 99,
 '§': 100

In [86]:
#handling speacial token allowed_special
#u can fork cl100k_base and add a new special token

**minbpe**

minbpe/exercice.py

In [87]:
#TikToken tokenizer library used by GPT-3.5, GPT-4, GPT-4o, GPT-4.1, GPT-5-mini
#It is:Byte-Level BPE with many improvements
import tiktoken
#enc = tiktoken.get_encoding("gpt2")
enc = tiktoken.encoding_for_model("gpt-4o-mini")
ids = enc.encode("Hello world!")
print(ids)


[13225, 2375, 0]


In [88]:
print(enc.decode(ids))


Hello world!


In [89]:
#SentencePiece Tokenizer language-independent subword tokenizer.
# used for both training and inference
import sentencepiece as spm


#In th Tik token we first take our code points in the string and
#we encode them used utf-8 to bytes and we're merging bytes

# vs

#for SentencePiece it works directly on the level of the code points themselves so
# it looks at whatever code points are available in the training set
# and start merging those code points and the BPE is running on the level of code points
#git mapped to a special unkown token or if u have a bite foldback option turned on

#  SentencePiece — Code Point–Level BPE

SentencePiece est un tokenizer BPE largement utilisé car **il peut à la fois s’entraîner et servir à l’inférence**, contrairement à TikToken (qui n’est pas conçu pour l’entraînement).

C’est le tokenizer officiel de nombreux modèles modernes :
- **LLaMA**  ,**Mistral** , **T5** , **ALBERT**  et d’autres

 **Repo officiel** : https://github.com/google/sentencepiece

---

## Fonctionnement : BPE sur les Points de Code Unicode

SentencePiece applique le BPE **directement sur les points de code Unicode**, et non sur les bytes UTF-8.  
Cela lui permet de travailler proprement sur toutes les langues, y compris celles **sans espaces** (comme le japonais ou le chinois).

---

##  Hyperparamètre : `character_coverage`

`character_coverage` détermine quels caractères doivent être intégrés au vocabulaire :

- Si un caractère est **fréquent**, il fait partie du vocab.
- Si un caractère est **rare** Unknown characters , deux cas :

###  1. Sans `byte_fallback`
→ Le caractère est remplacé par le token **`<unk>`** (unknown).

### 2. Avec `byte_fallback = true`
→ Le caractère rare est encodé en **UTF-8 bytes**, puis converti en **tokens byte**.

Cela garantit que tout caractère peut être encodé, même s'il est absent du vocabulaire.

---

##  Propriétés importantes

- Peut **fusionner des sous-mots** dans n’importe quelle langue
- Fonctionne dans les langues **sans espaces**
- Produit généralement des **séquences plus courtes** que les tokenizers byte-level
- Nécessite une gestion explicite des caractères inconnus :
  - soit `<unk>`
  - soit byte fallback



In [90]:
#text = "café"
# TikToken (byte-level)
# UTF-8 bytes: b'c' b'a' b'f' b'\xc3' b'\xa9'
# BPE merges are applied on bytes
# Could produce: ['c', 'a', 'f', 'é'] if merged or still byte tokens

# SentencePiece (code point-level)
# Code points: ['c', 'a', 'f', 'é'] (U+0063, U+0061, U+0066, U+00E9)
# BPE merges applied directly on code points
# Could produce: ['caf', 'é']
# Llama2 use SentencePiece

In [91]:
#in LM we want to keep the raw data as much as possible in a raw form
#si pour cela ils critiquent SentencePiece (normalisation)


In [92]:
import sentencepiece as spm
with open("toy.txt", "w", encoding="utf-8") as f:
  f.write("SentencePiece is an unsupervised text tokenizer and detokenizer mainly for Neural Network-based text generation systems where the vocabulary size is predetermined prior to the neural model training. SentencePiece implements subword units (e.g., byte-pair-encoding (BPE) [Sennrich et al.]) and unigram language model [Kudo.]) with the extension of direct training from raw sentences. SentencePiece allows us to make a purely end-to-end system that does not depend on language-specific pre/postprocessing.")

In [93]:
# train a sentencepiece model on it
# the settings here are (best effort) those used for training Llama 2
import os

options = dict(
  # input spec
  input="toy.txt",
  input_format="text",
  # output spec
  model_prefix="tok400", # output filename prefix
  # algorithm spec
  # BPE alg
  model_type="bpe",
  vocab_size=400,
  # normalization
  normalization_rule_name="identity", # ew, turn off normalization
  remove_extra_whitespaces=False,
  input_sentence_size=200000000, # max number of training sentences
  max_sentence_length=4192, # max number of bytes per sentence
  seed_sentencepiece_size=1000000,
  shuffle_input_sentence=True,
  # rare word treatment
  character_coverage=0.99995,
  byte_fallback=True,
  # merge rules
  split_digits=True,
  split_by_unicode_script=True,
  split_by_whitespace=True,
  split_by_number=True,
  max_sentencepiece_length=16,
  add_dummy_prefix=True,
  allow_whitespace_only_pieces=True,
  # special tokens
  unk_id=0, # the UNK token MUST exist
  bos_id=1, # the others are optional, set to -1 to turn off
  eos_id=2,
  pad_id=-1,
  # systems
  num_threads=os.cpu_count(), # use ~all system resources
)

spm.SentencePieceTrainer.train(**options)


In [94]:
sp = spm.SentencePieceProcessor()
sp.load('tok400.model')
vocab = [[sp.id_to_piece(idx), idx] for idx in range(sp.get_piece_size())]
vocab

[['<unk>', 0],
 ['<s>', 1],
 ['</s>', 2],
 ['<0x00>', 3],
 ['<0x01>', 4],
 ['<0x02>', 5],
 ['<0x03>', 6],
 ['<0x04>', 7],
 ['<0x05>', 8],
 ['<0x06>', 9],
 ['<0x07>', 10],
 ['<0x08>', 11],
 ['<0x09>', 12],
 ['<0x0A>', 13],
 ['<0x0B>', 14],
 ['<0x0C>', 15],
 ['<0x0D>', 16],
 ['<0x0E>', 17],
 ['<0x0F>', 18],
 ['<0x10>', 19],
 ['<0x11>', 20],
 ['<0x12>', 21],
 ['<0x13>', 22],
 ['<0x14>', 23],
 ['<0x15>', 24],
 ['<0x16>', 25],
 ['<0x17>', 26],
 ['<0x18>', 27],
 ['<0x19>', 28],
 ['<0x1A>', 29],
 ['<0x1B>', 30],
 ['<0x1C>', 31],
 ['<0x1D>', 32],
 ['<0x1E>', 33],
 ['<0x1F>', 34],
 ['<0x20>', 35],
 ['<0x21>', 36],
 ['<0x22>', 37],
 ['<0x23>', 38],
 ['<0x24>', 39],
 ['<0x25>', 40],
 ['<0x26>', 41],
 ['<0x27>', 42],
 ['<0x28>', 43],
 ['<0x29>', 44],
 ['<0x2A>', 45],
 ['<0x2B>', 46],
 ['<0x2C>', 47],
 ['<0x2D>', 48],
 ['<0x2E>', 49],
 ['<0x2F>', 50],
 ['<0x30>', 51],
 ['<0x31>', 52],
 ['<0x32>', 53],
 ['<0x33>', 54],
 ['<0x34>', 55],
 ['<0x35>', 56],
 ['<0x36>', 57],
 ['<0x37>', 58],
 ['<0x38>', 5

In [95]:
ids = sp.encode("hello 안녕하세요  مرحبا")
print(ids)


[362, 378, 361, 372, 358, 362, 239, 152, 139, 238, 136, 152, 240, 152, 155, 239, 135, 187, 239, 157, 151, 362, 362, 220, 136, 219, 180, 219, 176, 219, 171, 219, 170]


In [96]:
#decoding indiv tokens back into the little pieces
print([sp.id_to_piece(idx) for idx in ids])
#bytecode of unkown token
#qui n'appartiennent pas a notre training set
#  byte_fallback=False,
# we gonna get <unk> in the decoding
#llama use true
#white space = '_' and extra space in the begining because of   add_dummy_prefix=True,
# 'word' and  ' word' are different so we add spaces in begin


['▁', 'h', 'e', 'l', 'lo', '▁', '<0xEC>', '<0x95>', '<0x88>', '<0xEB>', '<0x85>', '<0x95>', '<0xED>', '<0x95>', '<0x98>', '<0xEC>', '<0x84>', '<0xB8>', '<0xEC>', '<0x9A>', '<0x94>', '▁', '▁', '<0xD9>', '<0x85>', '<0xD8>', '<0xB1>', '<0xD8>', '<0xAD>', '<0xD8>', '<0xA8>', '<0xD8>', '<0xA7>']


---
===

vocab_size

Q: what should be vocab size?

Q: how can I increase vocab size?

dans le code precedent ou on code mini_gpt from scratch on a self.

code transformeur

```
token_embedding_table = nn.Embedding(vocab_size, n_embd)

```
table d'emb 2_dim array vocab_size number of rows
each token has a vector that we're going to train using back propagation

the vector is of size n_emb= number of channels in the Transformer and basically as vocap size increases we're going to add rows in the table


lm_head(x) = nn.Linear(n_embd, vocab_size)to produce the logits = proba of the next token in the sequence





> Learning to Compress Prompts with Gist Tokens



**Soft Prompt Distillation: gist tokens**

y a une technique train the model by distillation so you are keeping the entire model frozen and you're only training the representations of *the new tokens* their embeddings and you're optimizing over the new tokens such taht the behavior of the language model is identical to the model that has a very long prompt


so it's a compression technique of compressing that very long prompt into those few new gist tokens

ni Laura ni finetuning you're training just token embeddings

---



[Let's build the GPT Tokenizer](https://https://www.youtube.com/watch?v=zduSFxRajkE)




> TAMING TRANSFORMERS FOR HIGH-RESOLUTION IMAGE SYNTHESIS

